In [ ]:
%matplotlib widget

import copy
import math
import matplotlib.pyplot as plt
import numpy as np
from lab1 import PIDSimulation, DELTA, show_value, get_exercise_params, plot_and_visualize, plot, visualize

# Enable constrained layout to make better use of blank space
plt.rcParams['figure.constrained_layout.use'] = True

class PID:
    def __init__(self, kp: float, ki: float, kd: float):
        self.kp = kp
        self.ki = ki
        self.kd = kd
        self.last_x = math.nan
        self.integral = 0

    def control(self, delta: float, x: float) -> float:
        vx = (x - self.last_x) / delta
        self.last_x = x
        self.integral = min(max(self.integral + x * self.ki * delta, -2), 2)
        # show_value('vx', vx)
        show_value('integral', self.integral)
        return -x * self.kp - vx * self.kd - self.integral

# TODO: Move this into exercise1?
# TODO: Get rid of explicit references to DELTA?
def simulate(pid: PID, params, max_time: int):
    sim = PIDSimulation(copy.deepcopy(pid), params)
    stable_time = 0
    while sim.t < max_time:
        sim.tick(DELTA)
        if abs(sim.vx) <= 0.001 and abs(sim.x) <= 0.001:
            stable_time += DELTA
            if stable_time > 1.0:
                return sim.t - stable_time
        else:
            stable_time = 0
    return math.inf

In [ ]:
plt.figure('Stabilization time', clear=True)
params = get_exercise_params(2)
x_data = np.arange(0, 10, 0.01)
y_data = [simulate(PID(10, 5, x), params=params, max_time=30) for x in x_data]
plt.plot(x_data, y_data)
plt.ylim(bottom=0)
plt.show()

In [ ]:
import scipy.optimize

def optimize(params):
    func = lambda x: simulate(PID(*x), params, max_time=30)
    bounds = [(0, 10), (0, 10), (0, 10)]
    return scipy.optimize.differential_evolution(func, bounds=bounds)

result = optimize(params=get_exercise_params(3))
result

In [ ]:
plot_and_visualize(PID(*result.x), get_exercise_params(3), max_time=60)

In [ ]:
from ipywidgets import interact, FloatSlider, Layout

fig = plt.figure('Interactive PID')

# Exercise parameters
interactive_params = get_exercise_params(1)

# Function that will be called every time the interactive inputs change
def plot_interactive(kp, ki, kd):
    pid = PID(kp, ki, kd)
    plot(pid, interactive_params, max_time=30, figure=fig)

# Start the interactive view with sliders for inputs
# See https://ipywidgets.readthedocs.io/en/latest/examples/Using%20Interact.html for more info
slider_layout = Layout(width='500px')
interact(
    plot_interactive,
    kp=FloatSlider(1, min=0, max=10, step=0.01, layout=slider_layout),
    ki=FloatSlider(0, min=0, max=10, step=0.01, layout=slider_layout),
    kd=FloatSlider(0, min=0, max=10, step=0.01,layout=slider_layout),
)

plt.show()